In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import numpy as np
import seaborn as sns
import re

In [ ]:
sns.set_palette('deep')

### Common methods

In [ ]:
types = ['random', 'bq', 'rq']

def load_results(pattern):
    dfs = []
    for comm_type in types:
        for filename in glob(pattern.format(comm_type)):
            dfi = pd.read_csv(filename, sep='\t')
            dfi['p'] = int(filename.split('.')[-3].split('_')[-1])
            dfi['type'] = comm_type
            dfs.append(dfi)
    return pd.concat(dfs)

In [ ]:
def sensitivity(df, kind):
    
    if kind == 'abiotic':
        df["initial"] = df["medium"] == 'minimal'
    
    if kind == 'biotic':
        expr = re.compile(".*_\d+_\d+")
        df["initial"] = df["community"].apply(lambda x: expr.match(x) is None)
        
    df_smt_sum = df.groupby(["type", "community", "p", "medium", "receiver", "donor", "initial"], 
                     as_index=False).agg({"smetana": np.sum})

    df0_smt_sum = df_smt_sum.query("initial == True").copy()
    df1_smt_sum = df_smt_sum.query("initial == False").copy()
    
    if kind == 'abiotic':
        df1_smt_sum["medium"] = df1_smt_sum["medium"].apply(lambda x: '_'.join(x.split('_')[:-1]))
    
    if kind == 'biotic':
        df1_smt_sum["community_old"] = df1_smt_sum["community"]
        df1_smt_sum["community"] = df1_smt_sum["community"].apply(lambda x: '_'.join(x.split('_')[:-1]))

    dX_all = pd.merge(df0_smt_sum, df1_smt_sum, how="left",
                      on = ["type", "community", "p", "medium", "receiver", "donor"],
                     ).fillna(0)
    
    dX_all["dX"] = ((dX_all["smetana_y"] - dX_all["smetana_x"])) ** 2

    dX_comm = dX_all.groupby(["type", "community", "p"], 
                                 as_index=False).agg({"dX":np.sum, "smetana_x": np.sum})
        
    dX_comm["sensitivity"] = dX_comm["dX"] / dX_comm["smetana_x"]

    return dX_comm

## Sensitivity to abiotic perturbations

In [ ]:
dfa = load_results("../simulation/abiotic/{}_subsample/*.tsv")

In [ ]:
dXa = sensitivity(dfa, 'abiotic')

In [ ]:
plt.figure(figsize=(9,3))

palette = {"random": '#cccccc', "bq": '#ed7e17', "rq": '#1ba055'}

sns.boxplot(data=dXa, x="p", y="sensitivity", hue="type", hue_order=types, 
            palette=palette, showfliers=False, linewidth=1)

ax = sns.stripplot(data=dXa, x="p", y="sensitivity", hue="type", hue_order=types, palette=palette, 
              dodge=True, size=3, linewidth=0.5, alpha=0.3)


plt.xlabel('Number of perturbations')
plt.ylabel('Sensitivity')
plt.ylim(0, 4.0)
plt.yticks([0,1,2,3,4])
ax.legend(ax.get_legend_handles_labels()[0], ["random", "cooperative", "competitive"])
plt.tight_layout()

plt.savefig("../figures/fig_5a.png", dpi=300)

## Sensitivity to biotic perturbations

In [ ]:
dfb = load_results("../simulation/biotic/{}_subsample/*.tsv")

In [ ]:
dXb = sensitivity(dfb, 'biotic')

In [ ]:
plt.figure(figsize=(9,3))

sns.boxplot(data=dXb, x="p", y="sensitivity", hue="type", hue_order=types, palette=palette,
            showfliers=False, linewidth=1)

ax = sns.stripplot(data=dXb, x="p", y="sensitivity", hue="type", hue_order=types, palette=palette, 
              dodge=True, size=3, linewidth=0.5, alpha=0.3)

plt.xlabel('Number of perturbations')
plt.ylabel('Sensitivity')
plt.ylim(0, 4.0)
plt.yticks([0,1,2,3,4])
ax.legend(ax.get_legend_handles_labels()[0], ["random", "cooperative", "competitive"])

plt.tight_layout()
plt.savefig("../figures/fig_5b.png", dpi=300)